In [ ]:
import asyncio
import requests
import os
import logging


directory = '/home/kalacheva/Desktop/big_redis/'

In [ ]:
class StubChecker:
    async def check(self, url):
        try:
            res = await asyncio.to_thread(
                requests.get,
                'http://blockchecker.ti.bi.zone/api/v1/check/',
                params={"value": url},
                verify=False,
            )
            return res.json()["block_status"]
        except Exception as e:
            logger.exception(e)
            return

In [ ]:
# example for one url

checker = StubChecker()
await checker.check(url)

url = 'http://adcz.hit.gemius.pl/'

In [ ]:
async def check_file(root, file):
    file_path = os.path.join(root, file)
    url = file[:-4]
    try:
        verdict = await asyncio.wait_for(checker.check(url), timeout=10)
    except asyncio.TimeoutError:
        verdict = False
    except NameError:
        verdict = False
    print(verdict, url, file_path)

    if verdict == True:
        os.remove(file_path)

async def main(directory):
    tasks = []
    for root, dirs, files in os.walk(directory):
#         print(len(files))
        for file in files:
            tasks.append(check_file(root, file))
    await asyncio.gather(*tasks)

In [ ]:
# iterate over all folders

await main(directory)